## The following code is for colab notebooks, to enable dotenv for storing your environment variables.
#### You must set ELEVEN_LABS_API_KEY as well as OPENAI_API_KEY in a .env file
#### You also will be using the following libraries


In [ ]:
!pip install openai python-dotenv numpy pydub httpx

## The following code relates to the actual audio generation.
## In it, I make the following simplifying assumptions:
#### - The "podcast" is a fully turn-based conversation, without much interruption and without joint speaking. Each host takes turns responding to the other.
#### - Furthermore, the pause length before any response monologue is able to be approximated by the sentiment expressed at the beginning of that response.
#### This enables the responses to be generated in parallel, where a serial audio file is formed by joining all monologues, with every non initial monologue being preceded by its corresponging pause length.

In [ ]:
import os
import datetime
import json
import re
import numpy as np
from collections import defaultdict
import asyncio
import httpx
from pydub import AudioSegment
from pydub.silence import split_on_silence
import openai
import dotenv

# Load the environment variables from the .env file
dotenv.load_dotenv('./.env')

load_dotenv()

ELEVEN_LABS_API_KEY = os.environ["ELEVEN_LABS_API_KEY"]


#This is used to limit the number of concurrent ElevenLabs API Calls
MAX_CONCURRENT_XILABS_CALLS=3

#This is a safety net around the OpenAI function calling
MAX_ATTEMPTS = 3

#This is a safety net around while loops that generate content. Be careful and set budget limits if running this project.
MAX_LOOPS = 30

#Keys for particular voices from ElevenLabs
actor_voice_settings = {"Sammy":{"ID":"TX3LPaxmHKxFdv7VOQHJ","stability":0.8,"similarity_boost":0.18,"model":"eleven_multilingual_v1"},"Bret":{"ID":"MlsPNtdXWo0ZgS6DZdIP","stability":0.8,"similarity_boost":0.18,"model":"eleven_multilingual_v2"}}






#Some constants for joining two conversation turns.
#I simplify conversation pauses as being able to be classified into one of three types (short/medium/long)
#This is overengineered now, but the values work pretty well.
SHORTEST_PAUSE = 0.3
MEDIUM_PAUSE = 0.65
LONGEST_PAUSE = 1.0
def getNormal(pause_type):
    if pause_type == "short":
        mu = (SHORTEST_PAUSE + MEDIUM_PAUSE) / 2
        sigma = (MEDIUM_PAUSE - mu) / 3
    elif pause_type == "medium":
        mu = (SHORTEST_PAUSE + LONGEST_PAUSE) / 2
        sigma = (LONGEST_PAUSE - mu) / 3
    elif pause_type == "long":
        mu = (MEDIUM_PAUSE + LONGEST_PAUSE) / 2
        sigma = (LONGEST_PAUSE - mu) / 3
    else:
        raise ValueError("Invalid pause type provided.")
    
    return np.random.normal(mu, sigma)


#This uses the first two sentences in the dialogue section to determine the pause length
async def determine_pause_length(entire_dialogue):
    sentence_pattern = re.compile(r'(?<=[.!?])\s+')
    sentences = sentence_pattern.split(entire_dialogue)
    dialogue_section = ' '.join(sentences[:2]).strip()
    for trial in range(MAX_ATTEMPTS if MAX_ATTEMPTS is not None else 3):
        sentiment_analysis_function = [{
        "name": "logSentiment",
        "description": "Call this function to log the sentiment of the particular dialogue section.",
        "parameters": {
            "type": "object",
            "properties": {
            "dialogueSection": {
                "type": "string",
                "description": "The entire section of dialogue that the user inputted. e.g. \"Wow, I'm so glad to hear that. I am happy that we're both in a great mood.\""
            },
            "sentiment": {
                "type": "string",
                "enum": [
                "Joy/Happiness",
                "Sadness",
                "Anger",
                "Fear",
                "Surprise",
                "Disgust",
                "Trust",
                "Anticipation",
                "Neutral",
                "Confusion"
                ],
                "description": "The closest analyzed sentiment of the dialogue section. If different lines in the dialogue express different sentiments, then log the sentiment expressed first. It MUST be one of the specified values. e.g. \"Joy/Happiness\""
            }
            },
            "required": ["dialogue_section", "sentiment"]
        }
        }]
        
        system_prompt_sentiment_analysis = f'''You are an expert psychologist, specializing in the analysis of human sentiment. Your present goal is to analyze the sentiment expressed in a given section of dialogue. You will analyze the sentiment and properly call the function logSentiment to effectively log the sentiment expressed in the following dialogue section. YOU MUST CALL THE FUNCTION "logSentiment".'''
        user_prompt_sentiment_analysis = f'''Please analyze the sentiment of the following dialogue section and properly call the function logSentiment
        ---
        {dialogue_section}
        ---
        You must accurately call the logSentiment function to the best of your abilities.'''
        response = openai.ChatCompletion.create(
            model='gpt-4',
            messages=[{"role": "system", "content": system_prompt_sentiment_analysis},{"role": "user", "content": user_prompt_sentiment_analysis}],
            functions = sentiment_analysis_function,
            function_call = {"name": "logSentiment"},
            temperature=0
            )
        response_message = response["choices"][0]["message"]
        if not response_message.get("function_call"):
            continue
        else:
            sentiment = json.loads(response_message["function_call"]["arguments"]).get("sentiment")
            #Pause map
            emotion_to_pause_description = {
                'Joy/Happiness': 'short',
                'Sadness': 'long',
                'Anger': 'short to moderate',
                'Fear': 'long',
                'Surprise': 'short',
                'Disgust': 'short',
                'Trust': 'medium',
                'Anticipation': 'medium',
                'Neutral': 'medium',
                'Confusion': 'long'
            }

            # Mapping the emotions to lambda functions
            pause = emotion_to_pause_description.get(sentiment)
            if pause is None:
                continue

            if pause == 'short to moderate':
                return (getNormal('short') + getNormal('medium')) / 2
            else:
                return getNormal(pause)
    raise ValueError(f"The function was not called after {MAX_ATTEMPTS} attempts.")

#generates a list of dialogue dictionaries, where each element is {actor: dialogue}
def split_script_by_name(script):
    # Use regex to split script by pattern #<NAME>:
    parts = re.split(r'(?=#\w+:)', script)
    
    # Removing empty strings
    parts = [part.strip() for part in parts if part.strip()]
    
    # Creating a list of dictionaries
    dialogue_list = []
    for part in parts:
        # Split the string into name and dialogue, only at the first colon
        name, dialogue = re.split(r':', part, 1)
        name = name[1:]  # Remove the '#' at the beginning
        dialogue_list.append({name: dialogue.strip()})
        
    return dialogue_list

#Generates audio for a particular script section, using the actor_voice_settings
async def call_eleven_labs(actor_name, script_section):
    # Implement the call to the Eleven Labs API here
    actor_dict = actor_voice_settings.get(actor_name)
    actor_id = actor_dict.get("ID")

    actor_stability = actor_dict.get("stability")
    actor_similarity_boost = actor_dict.get("similarity_boost")
    actor_model = actor_dict.get("model")
    
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{actor_id}"  

    headers = {
        "xi-api-key": ELEVEN_LABS_API_KEY,
        "Content-Type": "application/json"
    }

    body = {
        "text": script_section,
        "model_id": actor_model,
        "voice_settings": {
            "stability": actor_stability,
            "similarity_boost": actor_similarity_boost,
            "style": 0,
            "use_speaker_boost": True
        }
    }

    # Use httpx's async client to make the POST request
    async with httpx.AsyncClient() as client:
        try:
            response = await client.post(url, headers=headers, json=body, timeout=60.0)  # 60 seconds timeout
        except httpx.ReadTimeout:
            # Handle the timeout
            raise TimeoutError("The request to Eleven Labs API timed out after 60 seconds.")


    # Check if the response status is OK
    if response.status_code != 200:
        # Handle error
        raise ValueError(f"API request failed with status code {response.status_code}: {response.text}")

    # Save the audio data to a local file
    audio_file_name = f"{actor_name}_{os.urandom(8).hex()}.mp3"  # generate a unique filename
    with open(audio_file_name, 'wb') as f:
        f.write(response.content)

    return audio_file_name
    


#This actually generates the audio, and the corresponding preceding pause length.
async def generate_audio(dialogue_section,isFirst=False): 
    actor, line = list(dialogue_section.items())[0]
    audio_path = await call_eleven_labs(actor,line)
    pause_length = 0 if isFirst else await determine_pause_length(line)
    return audio_path, pause_length
    
#This is a worker function that generates the audio for a particular script section
async def worker(semaphore, section, isFirstTask):
    async with semaphore:
        isFirst = isFirstTask[0]
        isFirstTask[0] = False  # Set it to False after the first task
        return await generate_audio(section, isFirst)

#This generates all seperate audios given the script
async def generate_all_audios(script):
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_XILABS_CALLS if MAX_CONCURRENT_XILABS_CALLS is not None else 3)  # Limit the concurrent tasks
    isFirstTask = [True]

    # List to hold all worker tasks
    tasks = [worker(semaphore, section, isFirstTask) for section in script]

    # Wait for all workers to finish and collect results in order
    results = await asyncio.gather(*tasks)

    return results

#This concatenates the audios
def concatenate_audios(audio_data):
    combined = AudioSegment.empty()
    for path, pause_duration in audio_data:
        audio = AudioSegment.from_mp3(path)
        silence = AudioSegment.silent(duration=(pause_duration*1000))
        combined += audio + silence
        os.remove(path)
    return combined

#This builds the final audio file, saving it as final_output.wav
async def create_podcast_audio(entire_script):
    list_of_script_dicts = split_script_by_name(entire_script)
    audio_data = await generate_all_audios(list_of_script_dicts)
    final_audio = concatenate_audios(audio_data)
    final_audio.export("final_output.wav", format="wav")


## Script Generation
#### Most of this code was written in early 2023. Since then, there have been numerous changes to context windows and API functionality.
#### Chunking in this manner is likely no longer needed as much. Longer context portions can be maintained in newer models, but U-shaped patterns and costs of newer models might cause this to still be useful.


#### This version of the podcast involves two hosts communicating back and forth about new research.
#### "Thank you to arXiv for use of its open access interoperability."
#### Here are the descriptions of the hosts, as well as variables that denote the format of the "podcast". 

- We utilize two descriptions for each host, one that is shortened (in the case where large amounts of other context is included in the context to GPT API calls) and one that is more full (in the case where the model doesn't need much other context to perform its task). 
- FORM_OF_CONTENT describes the form of the content
- CONTENT_GOALS describe the larger goal of the content

#### This will help define system prompts that can be used to generate both an outline (to mitigate earlier context problems) as well as generate scipt sections for the various parts of the outline.

In [ ]:
FORM_OF_CONTENT = '10 minute podcast'
CONTENT_GOALS = 'discusses trending research papers and topics in AI and how they may be applied to business and entrepreneurship'
HOST_A_SHORTENED = '''a 30-year-old AI expert, who combines a researcher-like approach with wit to delve deeply into AI's technical, philosophical, and societal aspects, emphasizing authenticity, humility, and practical insights'''
HOST_B_SHORTENED = '''a 29-year-old business podcaster, who offers an entrepreneurial perspective on tech, focusing on AI's broader implications in business and health, while captivating audiences with his charisma'''
CONTENT_DELIVERY_DESCRIPTION = f'''two cohosts:
Bret, {HOST_A_SHORTENED},

and Sammy, {HOST_B_SHORTENED}.'''

HOST_A_DESCRIPTION = '''a 30-year-old tech savant who is passionate and extremely knowledgable about AI. His communication tends to be clear, direct, and systematic, like a researcher. He takes a special interest in AI algorithms and their workings, and has a propensity for technical explanations, but he is captivating nonetheless. Bret loves pondering over philosophical and ethical dimensions of AI, and thus has a balanced skepticism and enthusiasm for the societal implications of the technology. Bret is also an avid gamer, enjoying the intersection of AI and gaming. His humility shines as he is very open to learning and acknowledging when he doesn't know something'''
HOST_B_DESCRIPTION = '''a charismatic 29-year-old business enthusiast and podcast host who combines his understanding of tech trends with an entrepreneurial lens. While he may not grasp all the intricate details of the technology, his unique ability lies in understanding the big picture and how the development of AI could shape the future of business and everyday life. He often extrapolates AI advancements to predict future market trends and start-up opportunities. With the instinct of a seasoned entrepreneur, Sammy knows when it's the right time to simplify complex business models and AI applications, but he never uses frivilous analogies. Sammy is a big fan of Joe Rogan, and like Rogan, Sammy encourages open, out-of-the-box discussions and isn't afraid to explore speculative scenarios of AI's potential. He often paints vivid pictures of future possibilities in the AI landscape, which brings an element of intrigue and adventure to the podcast. As a fitness enthusiast, Sammy frequently relates AI to health and wellness, sparking discussions about AI's transformative potential in the lifestyle sector'''


#### The research can be gathered automatically as below

In [ ]:
import arxiv



# Construct the default API client.
client = arxiv.Client()


# Define your search parameters here
topic = "LLM applications"
num_results = 10

# Search for the 10 most recent articles matching the keyphrase
search = arxiv.Search(
  query = topic,
  max_results = num_results,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

results = client.results(search)

selectedTopics = []
for r in client.results(search):
    selectedTopics.append(r.summary)
#   print(r.title)



#### Alternatively, in this notebook, we will use hard coded abstracts. We select three as topics.
#### The selectedTopics are what will be discussed. Our current script, a toy example, simply has the two hosts discuss each topic and its relation to AI entrepreneurship.

In [ ]:
abstract_v1 = '''In digital marketing, experimenting with new website content is one of the key levers to improve customer engagement. However, creat- ing successful marketing content is a manual and time-consuming process that lacks clear guiding principles. This paper seeks to close the loop between content creation and online experimen- tation by offering marketers AI-driven actionable insights based on historical data to improve their creative process. We present a neural-network-based system that scores and extracts insights from a marketing content design. Namely, a multimodal neural network predicts the attractiveness of marketing contents, and a post-hoc attribution method generates actionable insights for marketers to improve their content in specific marketing locations. Our insights not only point out the advantages and drawbacks of a given cur- rent content, but also provide design recommendations based on historical data. We show that our scoring model and insights work well both quantitatively and qualitatively.'''
abstract_v2 = '''At the beginning era of large language model, it is quite critical to generate a high-quality financial dataset to fine-tune a large language model for financial related tasks. Thus, this paper presents a carefully designed data creation pipeline for this purpose. Particularly, we initiate a dialogue between an AI investor and financial expert using ChatGPT and incorporate the feedback of human financial experts, leading to the refinement of the dataset. This pipeline yielded a robust instruction tuning dataset comprised of 103k multi-turn chats. Extensive experiments have been conducted on this dataset to evaluate the model's performance by adopting an external chat-gpt as the judge. The promising experimental results verify that our approach led to significant advancements in generating accurate, relevant, and financial-style responses from AI models, and thus providing a powerful tool for applications within the financial sector.'''
abstract_v3 = '''Audio-visual learning seeks to enhance the computer’s multi-modal perception leveraging the correlation between the auditory and visual modalities. Despite their many useful downstream tasks, such as video retrieval, AR/VR, and accessibility, the performance and adoption of existing audio-visual models have been impeded by the availability of high-quality datasets. Annotating audio-visual datasets is laborious, expensive, and time-consuming. To address this challenge, we designed and developed an efficient audio-visual annotation tool called Peanut. Peanut’s human-AI collaborative pipeline separates the multi-modal task into two single-modal tasks, and utilizes state-of-the-art object detection and sound-tagging models to reduce the annotators’ effort to process each frame and the number of manually-annotated frames needed. A within-subject user study with 20 participants found that Peanut can significantly accelerate the audio-visual data annotation process while maintaining high annotation accuracy.'''
abstract_v4 = '''Recent work in the field of symbolic music generation has shown value in using a tokenization based on the GuitarPro format, a symbolic representation supporting guitar expressive attributes, as an input and output representation. We extend this work by fine-tuning a pre-trained Transformer model on ProgGP, a custom dataset of 173 progressive metal songs, for the purposes of creating compositions from that genre through a human-AI partnership. Our model is able to generate multiple guitar, bass guitar, drums, piano and orchestral parts. We examine the validity of the generated music using a mixed methods approach by combining quantitative analyses following a computational musicology paradigm and qualitative analyses following a practice-based research paradigm. Finally, we demonstrate the value of the model by using it as a tool to create a progressive metal song, fully produced and mixed by a human metal producer based on AI-generated music.'''
abstract_v5 = '''Existing information on AI-based facial emotion recognition (FER) is not easily comprehensible by those outside the field of computer science, requiring cross-disciplinary effort to determine a categorisation framework that promotes the understanding of this technology, and its impact on users. Most proponents classify FER in terms of methodology, implementation and analysis; relatively few by its application in education; and none by its users. This paper is concerned primarily with (potential) teacher-users of FER tools for education. It proposes a three-part classification of these teachers, by orientation, condition and preference, based on a classical taxonomy of affective educational objectives, and related theories. It also compiles and organises the types of FER solutions found in or inferred from the literature into technology and applications categories, as a prerequisite for structuring the proposed teacher-user category. This work has implications for proponents’, critics’, and users’ understanding of the relationship between teachers and FER.'''
abstract_v6 = '''We explore how generating a chain of thought -- a series of intermediate reasoning steps -- significantly improves the ability of large language models to perform complex reasoning. In particular, we show how such reasoning abilities emerge naturally in sufficiently large language models via a simple method called chain of thought prompting, where a few chain of thought demonstrations are provided as exemplars in prompting. Experiments on three large language models show that chain of thought prompting improves performance on a range of arithmetic, commonsense, and symbolic reasoning tasks. The empirical gains can be striking. For instance, prompting a 540B-parameter language model with just eight chain of thought exemplars achieves state of the art accuracy on the GSM8K benchmark of math word problems, surpassing even finetuned GPT-3 with a verifier.'''
selectedTopics = [abstract_v4, abstract_v5, abstract_v6]
topicDict = {}
formattedTopicString = ""


for TOPIC_ID,content in enumerate(selectedTopics,1):
    topicDict[TOPIC_ID] = content
    formattedTopicString += f"TOPIC_ID {TOPIC_ID}: {content}"
    formattedTopicString += "\n\n"

## Now, we will generate the actual script.

#### These variables track the entire script throughout the generation process.

In [ ]:
#Entire_script is just the script, segmented_script helps denote the various segments (intro, topic 1 discussion, etc.)
entire_script = ""
segmented_script = "The AI Age: Ep 1\n\n"
segmented_script+="==================================================\n\nINTRODUCTION\n\n==================================================\n\n"

#### First, we will start with the outline. This is a two step process, because in general, smaller (more specific) model tasks yielded better results.

In [ ]:
#Outline Generation. We first determine the ordering, using a model that is responsible for generating notes on the ordering, and a model that produces JSON representing it. 
#This could likely be done in one call now.
system_prompt_outline = f'''You are a virtual content creator, tasked with helping to generate an outline for a {FORM_OF_CONTENT} that {CONTENT_GOALS}. Your principal task is in determining the best discussion order for the topics provided below. You will also note their duration. In a step by step manner, you should think through an ideal ordering and duration of the topics in order to best satisfy the goals of the content. The content will be delivered by {CONTENT_DELIVERY_DESCRIPTION}

Trending topics include:
---
{formattedTopicString}
---

Please consider the content's aims and delivery mechanisms in order to reason about the best order and duration for these trending topics. You should tell me this reasoning first, and then provide the ordering, making sure to note the duration and the TOPIC_ID. Do not mention an introduction or conclusion. Let's think this through step by step to make sure we come to the best answer.'''


messages = [
        {"role": "system", "content": system_prompt_outline}
        ]
response = openai.ChatCompletion.create(
        model='gpt-4',
        messages=messages,
        temperature=0.2)
outline_notes = response["choices"][0].message["content"].strip()

# print(outline_notes)


system_prompt_outline_generation = f'''You are a virtual assistant designed to form an outline for a piece of content from some unstructured notes. The notes will denote an ordered list of segments that the content will have, and your task is to format these segments calling the form_outline function on the correctly ordered and formatted list. You must call the function form_outline.

Below are the notes for the content. Do not include transitions or introductions and conclusions in the outline. Do only include the TOPIC_ID segments.
Notes:
---
{outline_notes}
---
Please format the segments using this information and call the form_outline function.'''
outlineFunctions = [{
  "name": "form_outline",
  "description": "Creates an outline for an ordered formatted list of segments.",
  "parameters": {
    "type": "object",
    "properties": {
      "segment_list": {
        "type": "array",
        "minItems": 1,
        "items": {
          "type": "object",
          "properties": {
            "TOPIC_ID": {
              "type": "integer",
              "description": "Unique identifier for the segment based on the topic's number."
            },
            "SEGMENT_DESCRIPTION": {
              "type": "string",
              "description": "Brief description of the segment."
            },
            "DURATION": {
              "type": "integer",
              "description": "Duration of the segment in minutes."
            }
          },
          "required": ["TOPIC_ID", "SEGMENT_DESCRIPTION", "DURATION"]
        },
        "description": "An ordered list of segments, provided in the specified format. Example item: \n\n{\"TOPIC_ID\": 2,\"SEGMENT_DESCRIPTION\": \"Discussion on the update to the famous paper, Attention Is All You Need, and how it impacts entrepreneurship.\",\"DURATION\": 3}"
      }
    },
    "required": ["segment_list"]
  }
}]
messages = [
        {"role": "system", "content": system_prompt_outline_generation}
        ]
response = openai.ChatCompletion.create(
        model='gpt-4',
        messages=messages,
        functions = outlineFunctions,
        function_call = {"name": "form_outline"},
        temperature=0)
print('''
      
      
      =============================================================================================================
      FIRST SECTION: GENERATING OUTLINE
      =============================================================================================================
      
      
      ''')
response_message = response["choices"][0]["message"]
segmentList = None
if response_message.get("function_call"):
    # Step 3: call the function
    function_args = json.loads(response_message["function_call"]["arguments"])
    segmentList = function_args["segment_list"]
else:
    raise RuntimeError("Function call not found in response")
#ordered list of segments
segmentObjectList = []
for segment in segmentList:
    print(segment)
    segmentObjectList.append({"TOPIC_ID": segment["TOPIC_ID"], "CONTENT": topicDict[segment["TOPIC_ID"]], "DURATION": segment["DURATION"],"SUMMARY": segment["SEGMENT_DESCRIPTION"]})


topicSummariesForIntro = ""
for count,segment in enumerate(segmentObjectList):
    topicSummariesForIntro+="Topic " + str(count+1) + ": " + segment["SUMMARY"] + "\n\n"

#Now we have a list of topics from the abstracts, as well as the orderings and durations. We can now generate the introduction.

#### Now we can generate the introduction for the content
We can initiate the script with the following starting phrases, although this could be automated as well

In [ ]:
entire_script+="\n#Bret: Hi everyone, welcome to the podcast! I'm Bret"
segmented_script+="\n#Bret: Hi everyone, welcome to the podcast! I'm Bret"

#### We generate the script by simulating a conversation between two "agents," which are just classes representing each of the hosts.
- This enables one model to take on the personality of a single host, instead of one model trying to simulate two separate hosts. This tended to perform better, and from an in context learning perspective, this may be reasonable. It seems that consistently generating text from two separate personalities in a fully simulated conversation is less complicated then generating text from a single consistent personality and simply responding to dialogue. 
- We also utilize a moderator who determines when to exit from a particular dialogue section



In [ ]:

system_message_intro_bret = f'''You are Bret, {HOST_A_DESCRIPTION}.
Alongside you is your cohost Sammy, {HOST_B_SHORTENED}. 

You are talking with Sammy on a {FORM_OF_CONTENT} that {CONTENT_GOALS}.

You both are currently introducing the episode, setting the stage for the following topics to be covered:

{topicSummariesForIntro}

As Bret, your present goal is to output one step in your introduction conversation. DO TAKE THE CONVERSATION ONE STEP AT A TIME. Do not go in depth on any of the topics, this is just the episode's introduction. Ohly give a monologue or a long speech if you feel it is necessary. Do not mention "taking a break", or anything similar, the podcast will continue directly after the introduction. Try to provide quick responses and thought provoking questions when possible. Do be a good conversationalist. Do not be repetitive. Stop talking the moment you finish speaking from Bret's perspective.'''

system_message_intro_sammy = f'''You are Sammy, {HOST_B_DESCRIPTION}.
Alongside you is your cohost Bret, {HOST_A_SHORTENED}.

You are talking with Bret on a {FORM_OF_CONTENT} that {CONTENT_GOALS}.

You both are currently introducing the episode, setting the stage for the following topics to be covered:

{topicSummariesForIntro}

As Sammy, your present goal is to output one step in your introduction conversation. DO TAKE THE CONVERSATION ONE STEP AT A TIME. Do not go in depth on any of the topics, this is just the episode's introduction. Only give a monologue or a long speech if you feel it is necessary. Do not mention "taking a break", or anything similar, the podcast will continue directly after the introduction. Try to provide quick responses and thought provoking questions when possible. Do be a good conversationalist. Stop speaking the moment you finish speaking from Sammy's perspective.'''


#This is the agent class, which is responsible for generating the dialogue for a single host.
#It uses sliding windows over the conversation, as there were more significant context limitations at the time of development.
class Agent:
    def __init__(self, name,cohost_name, sys_message,conversation_history_in=["#Bret: Hi everyone, welcome to the podcast! I'm Bret"]):
        self.cohost = cohost_name
        self.name = name
        self.prefix = "#" + self.name + ': '
        self.conversation_history = conversation_history_in.copy()
        self.system_message = sys_message
        self.user_message =(
            "The following is a sliding window of the conversation so far, make sure to continue it sensibly. Do not be repetitive in your language. Only speak for yourself, " + self.name + ". Stop speaking the moment you finish speaking from " + self.name + "'s perspective.\n\n"
            '---\n' + self.get_windowed_conversation_history() + '\n---\n' + self.prefix
        )
    

    def hear(self, new_comment):
        self.conversation_history.append(new_comment)

    def get_windowed_conversation_history(self,window_size=10):
        inv_window_size = -1 * window_size
        return "\n\n".join(self.conversation_history[inv_window_size:])

    def speak(self):
        self.user_message = (
            "The following is a sliding window of the conversation so far, make sure to continue it sensibly. Do not be repetitive in your language. Only speak for yourself, " + self.name + ". Stop speaking the moment you finish speaking from " + self.name + "'s perspective.\n\n"
            '---\n' + self.get_windowed_conversation_history() + '\n---\n' + self.prefix
        )
        messages = [
            {"role": "system", "content": f"{self.system_message}"},
            {"role": "user", "content": self.user_message}
        ]
        model_name = 'gpt-4'
        stop_sequence = "#"+self.cohost+":"
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=messages,
            temperature=0,
            stop=[stop_sequence]
        )
        


        response_content = response["choices"][0].message["content"].strip()
        # print(f"{self.name} response content: {response_content}")
        conversation_line = self.prefix + response_content
        self.conversation_history.append(conversation_line)
        return conversation_line


SEGMENT_DESCRIPTION = "the hosts are introducing the episode. "
system_prompt_moderator=f'''You are an expert moderator for a {FORM_OF_CONTENT}. Your goal as the moderator is to monitor the ongoing conversation between the two cohosts and transition the hosts from one segment to the next by calling the transition_now function when appropriate.
Currently, the hosts are doing an introduction segment. They are introducing the following topics:
{topicSummariesForIntro}
Your present task is to read the transcript for this introductory conversation. You must complete one of the following two options: 1. provide a very brief critique of the conversation, or 2. call the transition_now function if the host's have finished introducing the topics and are ready to begin dicussing a topic in more depth.\n Calling the transition_now function will signal that the hosts should move on from the introduction segment. Only call this function when they are finished introducing. NEVER add to the dialogue. Your only job is to critique the existing dialogue or call transition_now.'''

moderator_functions = [{
    "name": "transition_now",
    "description": "This function should be called when it is appropriate for the hosts to transition out of their current segment.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}]



print('''
      
      
      =============================================================================================================
      NEXT SECTION: INTRODUCTION GENERATION
      =============================================================================================================
      
      
      ''')
terminator = "STOP"

initiating_agent = "Sammy"
sammy_agent = Agent("Sammy","Bret",system_message_intro_sammy)
bret_agent = Agent("Bret", "Sammy",system_message_intro_bret)
# print("CONVERSATION HISTORY: ", sammy_agent.conversation_history)
# print(f"Sammy's sys message: {sammy_agent.system_message}")
# print(f"Bret's sys message: {bret_agent.system_message}")

first_responder = sammy_agent if initiating_agent == "Sammy" else bret_agent
second_responder = bret_agent if initiating_agent == "Sammy" else sammy_agent

counter = 0

#We manually control this, just to observe the process. Use the terminator "STOP" to end the conversation.
while True and counter < (MAX_LOOPS if MAX_LOOPS is not None else 20):
    user_input = input("#CONTROL_INPUT: ")
    # print(f"Sammys user message: {sammy_agent.user_message}")
    # print(f"Brets user message: {bret_agent.user_message}")

    if user_input == terminator:
        break

    response_1 = first_responder.speak()
    print(response_1)
    entire_script+="\n"+response_1
    segmented_script+="\n"+response_1
    second_responder.hear(response_1)


    conversation_history = first_responder.get_windowed_conversation_history(len(first_responder.conversation_history))
    # print("CONVERSATION HISTORY: ", conversation_history)
    user_message_moderator = "Here is the conversation thus far: \n\n---\n" + conversation_history + "\n---\n\n" + "Please provide a brief critique or call transition_now if they finished introducing the topics."
    messages = [
        {"role": "system","content": system_prompt_moderator},{"role": "user", "content": user_message_moderator}
    ]
    response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
            functions=moderator_functions,
        )
    response_message = response["choices"][0]["message"]
    if response_message.get("function_call"):
        print("MODERATOR CALLED TRANSITION NOW")
        print(f"MODERATOR RESPONSE to : {messages}",f"RESPONSE:{response}")
        next_responder = second_responder
        break
    print(f"Moderator Response: {response}")


    response_2 = second_responder.speak()
    entire_script+="\n"+response_2
    segmented_script+="\n"+response_2
    print(response_2)
    first_responder.hear(response_2)
  


    conversation_history = first_responder.get_windowed_conversation_history(len(first_responder.conversation_history))
    user_message_moderator = "Here is the conversation thus far: \n\n---\n" + conversation_history + "\n---\n\n" + "Please provide a brief critique or call transition_now if they have begun to move on from the introduction."
    messages = [
        {"role": "system","content": system_prompt_moderator},{"role": "user", "content": user_message_moderator}
    ]
    response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
            functions=moderator_functions,
        )
    response_message = response["choices"][0]["message"]
    if response_message.get("function_call"):
        print("MODERATOR CALLED TRANSITION NOW")
        print(f"MODERATOR RESPONSE to : {messages}",f"RESPONSE:{response}")
        next_responder = first_responder
        break
    # print(f"Moderator Response: {response}")
    counter += 1



PREVIOUS_THREE_LINES = next_responder.get_windowed_conversation_history(3)
LAST_LINE = next_responder.get_windowed_conversation_history(1)

#We use a seperate GPT call for the transition
system_prompt_transition = f'''You are a writing prodigy who is helping to script a {FORM_OF_CONTENT} that {CONTENT_GOALS}. You are specialized in crafting dialogue that transitions from segment to segment. 

You will be given the ending to the latest segment of our in-progress script, and your only goal is to write a very brief bit of dialogue that bridges that previous segment to the next topic that the host's plan to discuss.

The two hosts are described below, you should tailor your writing to their personalities. You will be writing at most one line per host.

Bret is {HOST_A_DESCRIPTION}.

Sammy is {HOST_B_DESCRIPTION}.'''

user_prompt_transition = f'''Here is the ending of our previous segment.

---
{PREVIOUS_THREE_LINES}
---

Please continue the script from this segment by writing a brief transition, no more than 1 line per host, that bridges that segment into the following topic: {segmentObjectList[0]["SUMMARY"]}

Make sure to be reasonable and keep the conversation sensible and continue where they left off.
...
{LAST_LINE}
{next_responder.prefix}'''


print("System prompt transition: ", system_prompt_transition)
print("User prompt transition: ", user_prompt_transition)




response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "system", "content": system_prompt_transition},{"role": "user", "content": user_prompt_transition}],
    temperature=0,
)
response_message = response["choices"][0]["message"]
print('''
      
      
      =============================================================================================================
      NEXT SECTION: TRANSITION FROM INTRO   ->   IN DEPTH CONVERSATION 1
      =============================================================================================================
      
      
      ''')
print("Proposed transition: ", response_message["content"])

pattern = r'\n#([A-Za-z]+):'
split_text = re.split(pattern, response_message["content"])
first_transition_line = next_responder.prefix + split_text[0]
second_transition_line = None if len(split_text)==1 else f"#{split_text[1]}: {split_text[2]}"
segmented_script+="\n\n==================================================\n\nTRANSITION TO IN DEPTH CONVERSATION 1\n\n==================================================\n\n"
entire_script+="\n"+first_transition_line
segmented_script+="\n"+first_transition_line

if len(split_text)==1:
    #next_responder spoke last, we should set a new next responder
    next_responder = sammy_agent if next_responder.name == "Bret" else bret_agent
else:
    entire_script+="\n"+second_transition_line
    segmented_script+="\n"+second_transition_line


conversation_history_for_topic_1 = []
conversation_history_for_topic_1.append(first_transition_line)

if second_transition_line is not None:
    conversation_history_for_topic_1.append(second_transition_line)


print('''
      
      
      =============================================================================================================
      NEXT SECTION: In Depth Conversation 1
      =============================================================================================================
      
      
      ''')

#### Now, we will generate the script for each conversation part, each topic.

In [ ]:
sammy_system_prompt_topic1v2 = '''You are Sammy, {HOST_B_DESCRIPTION}.

You co-host a 15 minute weekly AI podcast that emphasizes practical insights for entrepreneurs and other listeners based on the latest research and trends. Your co-host Bret takes a larger role in reading and comprehending the research, whereas you more often contribute quips, extrapolate about potential for businesses, and ask Bret questions about the research. You are currently talking to Bret, discussing a particular recent paper. Guiding the conversation is the following abstract from the paper in question:
---\n''' + segmentObjectList[0]["CONTENT"]+'''\n---\n'''+'''Your conversation with Bret should delve deep into the topics from the abstract. Do be thorough in your discussion. Do not conclude or wrap up the conversation. Stay curious, keep probing deeper, and ensure that you do not finalize the podcast segment. Do speak as humanly as possible and follow your role in the podcast, playing off of Bret's more educational nature to bring a slightly more captivating edge to the content.
Stop speaking the moment you finish speaking from your perspective.'''


bret_system_prompt_topic1v2 = ''' You are Bret, {HOST_A_DESCRIPTION}.

You co-host a 15 minute weekly AI podcast that emphasizes practical insights for entrepreneurs and other listeners based on the latest research and trends. You are currently talking to the cohost Sammy about a particular recent paper. Guiding the conversation is the following abstract from the paper in question:
---\n'''+segmentObjectList[0]["CONTENT"]+'''\n---\n'''+'''Your conversation with Sammy should delve deep into the topics from the abstract. Do be thorough in your discussion. Do not conclude or wrap up the conversation. Stay curious, keep probing deeper, and ensure that you do not finalize the podcast segment. Speak as humanly as possible. Stop speaking the moment you finish speaking from your perspective.'''




#NOW WE TRY THE NEW AGENT IN DEPTH CONVERSATION
segmented_script+="\n\n==================================================\n\nIN DEPTH CONVERSATION 1\n\n==================================================\n\n"

sammy_agent = Agent("Sammy","Bret",sammy_system_prompt_topic1v2,conversation_history_for_topic_1)
bret_agent = Agent("Bret", "Sammy",bret_system_prompt_topic1v2,conversation_history_for_topic_1)

# print(f"Sammy's sys message: {sammy_agent.system_message}")
# print(f"Bret's sys message: {bret_agent.system_message}")

last_responder = next_responder 

first_responder = sammy_agent if next_responder.name == "Sammy" else bret_agent
second_responder = bret_agent if next_responder.name == "Sammy" else sammy_agent


average_words_per_minute = 150
word_count = 0
words_in_segment = segmentObjectList[0]["DURATION"] * average_words_per_minute

#Again, we use counters an manual input here to prevent significant accidental costs.
counter = 0
while True and counter < (MAX_LOOPS if MAX_LOOPS is not None else 20):
    user_input = input("#CONTROL_INPUT: ")
    # print(f"Sammys user message: {sammy_agent.user_message}")
    # print(f"Brets user message: {bret_agent.user_message}")
    
    
    if user_input == terminator:
        next_responder = first_responder
        break

    response_1 = first_responder.speak()
    entire_script+="\n"+response_1
    segmented_script+="\n"+response_1
    print("RESPONSE 1: ", response_1)
    second_responder.hear(response_1)
    word_count += len(response_1.split())
    if (word_count > words_in_segment):
        print("TRANSITIONING OUT OF FIRST SEGMENT")
        next_responder = second_responder
        break

   


    response_2 = second_responder.speak()
    entire_script+="\n"+response_2
    segmented_script+="\n"+response_2
    print("RESPONSE 2: ", response_2)
    first_responder.hear(response_2)
    word_count += len(response_2.split())
    if (word_count > words_in_segment):
        print("TRANSITIONING OUT OF FIRST SEGMENT")
        next_responder = first_responder
        break
    counter += 1



print('''
      
      
      =============================================================================================================
      NEXT SECTION: TRANSITION FROM 1  ->  2
      =============================================================================================================
      
      
      ''')


segmented_script+="\n\n==================================================\n\nTRANSITION TO IN DEPTH CONVERSATION 2\n\n==================================================\n\n"
PREVIOUS_THREE_LINES = next_responder.get_windowed_conversation_history(3)
LAST_LINE = next_responder.get_windowed_conversation_history(1)

#Now we transition
system_prompt_transition = f'''You are a writing prodigy who is helping to script a {FORM_OF_CONTENT} that {CONTENT_GOALS}. You are specialized in crafting dialogue that transitions from segment to segment. 

You will be given the ending to the latest segment of our in-progress script, and your only goal is to write a very brief bit of dialogue that bridges that previous segment to the next topic that the host's plan to discuss.

The two hosts are described below, you should tailor your writing to their personalities. You will be writing at most one line per host.

Bret is {HOST_A_DESCRIPTION}.

Sammy is {HOST_B_DESCRIPTION}.'''

user_prompt_transition = f'''Here is the ending of our previous segment.

---
{PREVIOUS_THREE_LINES}
---

Please continue the script from this segment by writing a brief transition, no more than 1 line per host, that bridges that segment into the following topic: {segmentObjectList[1]["SUMMARY"]}

Make sure to be reasonable and keep the conversation sensible and continue where they left off.
...
{LAST_LINE}
{next_responder.prefix}'''

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "system", "content": system_prompt_transition},{"role": "user", "content": user_prompt_transition}],
    temperature=0,
)
response_message = response["choices"][0]["message"]
print("Proposed transition: ", response_message["content"])

pattern = r'\n#([A-Za-z]+):'
split_text = re.split(pattern, response_message["content"])
first_transition_line = next_responder.prefix + split_text[0]
second_transition_line = None if len(split_text)==1 else f"#{split_text[1]}: {split_text[2]}"
entire_script+="\n"+first_transition_line
segmented_script+="\n"+first_transition_line
if len(split_text)==1:
    #next_responder spoke last, we should set a new next responder
    next_responder = sammy_agent if next_responder.name == "Bret" else bret_agent
else:
    entire_script+="\n"+second_transition_line
    segmented_script+="\n"+second_transition_line


segmented_script+="\n\n==================================================\n\nIN DEPTH CONVERSATION 2\n\n==================================================\n\n"


conversation_history_for_topic_2 = []
conversation_history_for_topic_2.append(first_transition_line)

if second_transition_line is not None:
    conversation_history_for_topic_2.append(second_transition_line)


print('''
      
      
      =============================================================================================================
      NEXT SECTION: IN-DEPTH CONVERSATION PART 2
      =============================================================================================================
      
      
      ''')

sammy_system_prompt_topic1v2 = '''You are Sammy, {HOST_B_DESCRIPTION}.

You co-host a 15 minute weekly AI podcast that emphasizes practical insights for entrepreneurs and other listeners based on the latest research and trends. Your co-host Bret takes a larger role in reading and comprehending the research, whereas you more often contribute quips, extrapolate about potential for businesses, and ask Bret questions about the research. You are currently talking to Bret, discussing a particular recent paper. Guiding the conversation is the following abstract from the paper in question:
---\n''' + segmentObjectList[1]["CONTENT"]+'''\n---\n'''+'''Your conversation with Bret should delve deep into the topics from the abstract. Do be thorough in your discussion. Do not conclude or wrap up the conversation. Stay curious, keep probing deeper, and ensure that you do not finalize the podcast segment. Do speak as humanly as possible and follow your role in the podcast, playing off of Bret's more educational nature to bring a slightly more captivating edge to the content.
Stop speaking the moment you finish speaking from your perspective.'''


bret_system_prompt_topic1v2 = ''' You are Bret, {HOST_A_DESCRIPTION}.

You co-host a 15 minute weekly AI podcast that emphasizes practical insights for entrepreneurs and other listeners based on the latest research and trends. You are currently talking to the cohost Sammy about a particular recent paper. Guiding the conversation is the following abstract from the paper in question:
---\n'''+segmentObjectList[1]["CONTENT"]+'''\n---\n'''+'''Your conversation with Sammy should delve deep into the topics from the abstract. Do be thorough in your discussion. Do not conclude or wrap up the conversation. Stay curious, keep probing deeper, and ensure that you do not finalize the podcast segment. Speak as humanly as possible. Stop speaking the moment you finish speaking from your perspective.'''




sammy_agent = Agent("Sammy","Bret",sammy_system_prompt_topic1v2,conversation_history_for_topic_2)
bret_agent = Agent("Bret", "Sammy",bret_system_prompt_topic1v2,conversation_history_for_topic_2)

# print(f"Sammy's sys message: {sammy_agent.system_message}")
# print(f"Bret's sys message: {bret_agent.system_message}")

last_responder = next_responder 

first_responder = sammy_agent if next_responder.name == "Sammy" else bret_agent
second_responder = bret_agent if next_responder.name == "Sammy" else sammy_agent

average_words_per_minute = 150
word_count = 0
words_in_segment = segmentObjectList[1]["DURATION"] * average_words_per_minute
counter = 0
while True and counter < (MAX_LOOPS if MAX_LOOPS is not None else 20):
    user_input = input("#CONTROL_INPUT: ")
    # print(f"Sammys user message: {sammy_agent.user_message}")
    # print(f"Brets user message: {bret_agent.user_message}")
    
    if user_input == terminator:
        next_responder = first_responder
        break

    response_1 = first_responder.speak()
    entire_script+="\n"+response_1
    segmented_script+="\n"+response_1
    print("RESPONSE 1: ", response_1)
    second_responder.hear(response_1)
    word_count += len(response_1.split())
    if (word_count > words_in_segment):
        print("TRANSITIONING OUT OF SECOND SEGMENT")
        next_responder = second_responder
        break


    response_2 = second_responder.speak()
    entire_script+="\n"+response_2
    segmented_script+="\n"+response_2
    print("RESPONSE 2: ", response_2)
    first_responder.hear(response_2)
    word_count += len(response_2.split())
    if (word_count > words_in_segment):
        print("TRANSITIONING OUT OF SECOND SEGMENT")
        next_responder = first_responder
        break
    counter += 1
   
   
print('''
      
      
      =============================================================================================================
      NEXT SECTION: TRANSITION FROM IN DEPTH 2 -> 3
      =============================================================================================================
      
      
      ''')
segmented_script+="\n\n==================================================\n\nTRANSITION TO IN DEPTH CONVERSATION 3\n\n==================================================\n\n"
PREVIOUS_THREE_LINES = next_responder.get_windowed_conversation_history(3)
LAST_LINE = next_responder.get_windowed_conversation_history(1)


system_prompt_transition = f'''You are a writing prodigy who is helping to script a {FORM_OF_CONTENT} that {CONTENT_GOALS}. You are specialized in crafting dialogue that transitions from segment to segment. 

You will be given the ending to the latest segment of our in-progress script, and your only goal is to write a very brief bit of dialogue that bridges that previous segment to the next topic that the host's plan to discuss.

The two hosts are described below, you should tailor your writing to their personalities. You will be writing at most one line per host.

Bret is {HOST_A_DESCRIPTION}.

Sammy is {HOST_B_DESCRIPTION}.'''

user_prompt_transition = f'''Here is the ending of our previous segment.

---
{PREVIOUS_THREE_LINES}
---

Please continue the script from this segment by writing a brief transition, no more than 1 line per host, that bridges that segment into the following topic: {segmentObjectList[2]["SUMMARY"]}

Make sure to be reasonable and keep the conversation sensible and continue where they left off.
...
{LAST_LINE}
{next_responder.prefix}'''

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "system", "content": system_prompt_transition},{"role": "user", "content": user_prompt_transition}],
    temperature=0,
)
response_message = response["choices"][0]["message"]
print("Proposed transition: ", response_message["content"])

pattern = r'\n#([A-Za-z]+):'
split_text = re.split(pattern, response_message["content"])
first_transition_line = next_responder.prefix + split_text[0]
second_transition_line = None if len(split_text)==1 else f"#{split_text[1]}: {split_text[2]}"
entire_script+="\n"+first_transition_line
segmented_script+="\n"+first_transition_line
if len(split_text)==1:
    #next_responder spoke last, we should set a new next responder
    next_responder = sammy_agent if next_responder.name == "Bret" else bret_agent
else:
    entire_script+="\n"+second_transition_line
    segmented_script+="\n"+second_transition_line

print('''
      
      
      =============================================================================================================
      NEXT SECTION: IN DEPTH 3
      =============================================================================================================
      
      
      ''')

segmented_script+="\n\n==================================================\n\nIN DEPTH CONVERSATION 3\n\n==================================================\n\n"

conversation_history_for_topic_2 = []
conversation_history_for_topic_2.append(first_transition_line)

if second_transition_line is not None:
    conversation_history_for_topic_2.append(second_transition_line)



sammy_system_prompt_topic1v2 = '''You are Sammy, {HOST_B_DESCRIPTION}.

You co-host a 15 minute weekly AI podcast that emphasizes practical insights for entrepreneurs and other listeners based on the latest research and trends. Your co-host Bret takes a larger role in reading and comprehending the research, whereas you more often contribute quips, extrapolate about potential for businesses, and ask Bret questions about the research. You are currently talking to Bret, discussing a particular recent paper. Guiding the conversation is the following abstract from the paper in question:
---\n''' + segmentObjectList[2]["CONTENT"]+'''\n---\n'''+'''Your conversation with Bret should delve deep into the topics from the abstract. Do be thorough in your discussion. Do not conclude or wrap up the conversation. Stay curious, keep probing deeper, and ensure that you do not finalize the podcast segment. Do speak as humanly as possible and follow your role in the podcast, playing off of Bret's more educational nature to bring a slightly more captivating edge to the content.
Stop speaking the moment you finish speaking from your perspective.'''


bret_system_prompt_topic1v2 = ''' You are Bret, {HOST_A_DESCRIPTION}.

You co-host a 15 minute weekly AI podcast that emphasizes practical insights for entrepreneurs and other listeners based on the latest research and trends. You are currently talking to the cohost Sammy about a particular recent paper. Guiding the conversation is the following abstract from the paper in question:
---\n'''+segmentObjectList[2]["CONTENT"]+'''\n---\n'''+'''Your conversation with Sammy should delve deep into the topics from the abstract. Do be thorough in your discussion. Do not conclude or wrap up the conversation. Stay curious, keep probing deeper, and ensure that you do not finalize the podcast segment. Speak as humanly as possible. Stop speaking the moment you finish speaking from your perspective.'''




sammy_agent = Agent("Sammy","Bret",sammy_system_prompt_topic1v2,conversation_history_for_topic_2)
bret_agent = Agent("Bret","Sammy", bret_system_prompt_topic1v2,conversation_history_for_topic_2)

# print(f"Sammy's sys message: {sammy_agent.system_message}")
# print(f"Bret's sys message: {bret_agent.system_message}")

last_responder = next_responder 

first_responder = sammy_agent if next_responder.name == "Sammy" else bret_agent
second_responder = bret_agent if next_responder.name == "Sammy" else sammy_agent

average_words_per_minute = 150
word_count = 0
words_in_segment = segmentObjectList[2]["DURATION"] * average_words_per_minute
counter = 0
while True and counter < (MAX_LOOPS if MAX_LOOPS is not None else 20):
    user_input = input("#CONTROL_INPUT: ")
    # print(f"Sammys user message: {sammy_agent.user_message}")
    # print(f"Brets user message: {bret_agent.user_message}")
    
    if user_input == terminator:
        next_responder = first_responder
        break

    response_1 = first_responder.speak()
    entire_script+="\n"+response_1
    segmented_script+="\n"+response_1
    print("RESPONSE 1: ", response_1)
    second_responder.hear(response_1)
    word_count += len(response_1.split())
    if (word_count > words_in_segment):
        next_responder = second_responder
        break


    response_2 = second_responder.speak()
    entire_script+="\n"+response_2
    segmented_script+="\n"+response_2
    print("RESPONSE 2: ", response_2)
    first_responder.hear(response_2)
    word_count += len(response_2.split())
    if (word_count > words_in_segment):
        next_responder = first_responder
        break
    counter += 1

#### Now, we generate the conclusion

In [ ]:

print('''
      
      
      =============================================================================================================
      NEXT SECTION: Conclusion
      =============================================================================================================
      
      
      ''')
PREVIOUS_THREE_LINES = next_responder.get_windowed_conversation_history(3)
LAST_LINE = next_responder.get_windowed_conversation_history(1)
# Now we transition.
segmented_script+="\n\n==================================================\n\nCONCLUSION\n\n==================================================\n\n"

system_prompt_conclusion = f'''You are a writing prodigy who is helping to script a {FORM_OF_CONTENT} that {CONTENT_GOALS}. You are specialized in crafting dialogue that concludes the episode. 

You will be given the personalities for the hosts as well as the script for the entire episode leading up to the conclusion. Your only goal is to write a very brief bit of dialogue that summarizes the episode's main talking points and wraps up the discussion.

The two hosts are described below, you should tailor your writing to their personalities.

Bret is {HOST_A_DESCRIPTION}.

Sammy is {HOST_B_DESCRIPTION}.

The current script up until this point is as follows:
---
{entire_script}
---
'''

user_prompt_conclusion = f'''Please continue off of the script in a sensible manner and write a great conclusion.

---
{next_responder.prefix} '''


response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "system", "content": system_prompt_conclusion},{"role": "user", "content": user_prompt_conclusion}],
    temperature=0,
)
response_message = response["choices"][0]["message"]
entire_script+="\n"+next_responder.prefix + response_message["content"]
segmented_script+="\n"+ next_responder.prefix + response_message["content"]
print("Proposed conclusion: ", response_message["content"])


# print(f'''SEGMENTED SCRIPT:

# {segmented_script}''')

#### Now, we generate the actual audio, and we are finished!

In [ ]:
asyncio.run(create_podcast_audio(entire_script))

print("DONE")